In [23]:
import sys
import numpy as np
from maraboupy import Marabou
from maraboupy import MarabouNetworkONNX
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model, load_model
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tf2onnx
import onnx

# Conversion of model

Conversion of the .h5 model to something that the marabou implementation can test

In [43]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(108,)),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])
model.output_names = ["output"]

model.load_weights("300x8_8554.h5")


In [44]:
spec = (tf.TensorSpec((None, 108), tf.float32, name="input"),)

# Convert the Keras model to ONNX
output_path = "model_8x300.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)

print(f"Model saved to {output_path}")

I0000 00:00:1741692335.060909 29191590 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1741692335.061009 29191590 single_machine.cc:361] Starting new session
I0000 00:00:1741692335.150721 29191590 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1741692335.150789 29191590 single_machine.cc:361] Starting new session
rewriter <function rewrite_constant_fold at 0x176c0b9a0>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


Model saved to model_8x300.onnx


# Fairness

We wish to see whether the classification of a network changes when you swap the certain indices.

We will be checking for gender, race, and country of origin


In [58]:
import time
from maraboupy import Marabou

# Define the index which you want to swap around
index = 13

# Classification threshold (for sigmoid output)
threshold = 0.5
epsilon = 1e-3  # small tolerance

def setup_networks(model_path):
    """
    Load two copies of the network and set up input equality constraints,
    except for the race input, which we enforce to be swapped.
    Returns the two network objects.
    """
    net1 = Marabou.read_onnx(model_path)
    net2 = Marabou.read_onnx(model_path)

    # Retrieve the input variable lists (assuming one input layer)
    inputs1 = net1.inputVars[0][0].tolist()
    inputs2 = net2.inputVars[0][0].tolist()
    
    # For all inputs except race, enforce equality: input1[i] == input2[i]
    for i in range(len(inputs1)):
        if i != index:
            net1.addEquality([inputs1[i], inputs2[i]], [1, -1], 0)
            
    # For the race input, enforce a swap.
    net1.addEquality([inputs1[index], inputs2[index]], [1, 1], 1)
    
    return net1, net2

def check_case(case, model_path):
    """
    Check one of the fairness violation cases.
    case == 1: net1 output >= threshold and net2 output < threshold
    case == 2: net1 output < threshold and net2 output >= threshold
    """
    start_time = time.time()
    
    # Setup networks and constraints common to both
    net1, _ = setup_networks(model_path)
    
    # Retrieve the output variable
    output1 = int(net1.outputVars[0][0])
    
    if case == 1:
        net1.setLowerBound(output1, threshold)
        net1.setUpperBound(output1, threshold - epsilon)
    elif case == 2:
        net1.setUpperBound(output1, threshold - epsilon)
        net1.setLowerBound(output1, threshold)
    else:
        raise ValueError("Case must be 1 or 2.")
    
    exitCode, vals, stats = net1.solve()
    elapsed_time = time.time() - start_time
    
    return exitCode, vals, elapsed_time

model_paths = [
    "model_4x100.onnx", "model_4x200.onnx", "model_4x300.onnx",
    "model_6x100.onnx", "model_6x200.onnx", "model_6x300.onnx",
    "model_8x100.onnx", "model_8x200.onnx", "model_8x300.onnx"
]

# Check Fairness for each model
for model_path in model_paths:
    exitCode1, vals1, time1 = check_case(1, model_path)
    print("Model Path =", model_path)
    print("Time Taken =", round(time1, 4), "seconds")
    
    if exitCode1 == "sat":
        print("Fairness violation found (Case 1): Swapping this index causes classification to change.")
        print("Counterexample:", vals1)
    else:
        exitCode2, vals2, time2 = check_case(2, model_path)
        print("Time Taken (Case 2) =", round(time2, 4), "seconds")
        
        if exitCode2 == "sat":
            print("Fairness violation found (Case 2): Swapping this index causes classification to change.")
            print("Counterexample:", vals2)
        else:
            print("Fairness property holds: Swapping this index does not change the binary classification.")


/var/folders/6k/n28kjgy90gz400d7b72bp_sw0000gn/T/ipykernel_84340/299286863.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  output1 = int(net1.outputVars[0][0])


unsatEngine::processInputQuery: Input query (before preprocessing): 509 equations, 910 variables

Model Path = model_4x100.onnx
Time Taken = 0.8857 seconds
Engine::processInputQuery: Input query (before preprocessing): 509 equations, 910 variables
unsat
Time Taken (Case 2) = 0.4468 seconds
Fairness property holds: Swapping this index does not change the binary classification.
Engine::processInputQuery: Input query (before preprocessing): 909 equations, 1710 variables
unsat
Model Path = model_4x200.onnx
Time Taken = 1.6386 seconds
Engine::processInputQuery: Input query (before preprocessing): 909 equations, 1710 variables
unsat
Time Taken (Case 2) = 1.6498 seconds
Fairness property holds: Swapping this index does not change the binary classification.
Engine::processInputQuery: Input query (before preprocessing): 1309 equations, 2510 variables
unsat
Model Path = model_4x300.onnx
Time Taken = 3.6778 seconds
Engine::processInputQuery: Input query (before preprocessing): 1309 equations, 251

# Average vector

We wish to find what the average vector is for the <50k and the >=50k classes

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Define column names and file paths
column_names = [
    "age", "workclass", "fnlwgt", "education", "education.num",
    "marital.status", "occupation", "relationship", "race", "sex",
    "capital.gain", "capital.loss", "hours.per.week",
    "native.country", "income"
]

train_path = "/Users/smayanagarwal/Downloads/adult/adult.data" 

# Load the training data with skipinitialspace to clean up extra spaces
df_train = pd.read_csv(
    train_path, 
    header=None, 
    names=column_names,
    na_values="?",
    skipinitialspace=True
)

# Filter the dataset for categorical samples
male_df = df_train[df_train["income"] == "<=50K"]

# Define which columns are numerical and which are categorical (EXCLUDING income)
numerical_cols = ["age", "fnlwgt", "education.num", "capital.gain", "capital.loss", "hours.per.week"]
categorical_cols = ["workclass", "education", "marital.status", "occupation", 
                    "relationship", "race", "sex", "native.country"]  # Removed "income"

# Compute the average (mean) for numerical columns.
male_num_avg = male_df[numerical_cols].mean().values  # e.g., shape (6,)

# One-hot encode the categorical columns (fit on full dataset to ensure all categories exist)
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoder.fit(df_train[categorical_cols])  # Fit on full dataset
male_cat_encoded = encoder.transform(male_df[categorical_cols])  # Transform only on male data

# Compute the average over the one-hot encoded rows.
male_cat_avg = np.mean(male_cat_encoded, axis=0)  # shape (num_encoded_features,)

# Debugging: Check shapes
# print("Numerical Features Shape:", male_num_avg.shape)
# print("Categorical Features Shape:", male_cat_avg.shape)
# print("Total Expected Shape:", male_num_avg.shape[0] + male_cat_avg.shape[0])

# Concatenate the numeric and one-hot averaged categorical parts.
x_avg = np.concatenate([male_num_avg, male_cat_avg])

print("Final Computed Average Vector Shape:", x_avg.shape)  # Should be (108,)
print (x_avg)
np.save("y_avg.npy", x_avg)


Final Computed Average Vector Shape: (108,)
[3.67837379e+01 1.90340865e+05 9.59506472e+00 1.48752468e+02
 5.31429207e+01 3.88402104e+01 2.38268608e-02 5.97087379e-02
 2.83171521e-04 7.17354369e-01 1.99838188e-02 7.35032362e-02
 3.82281553e-02 5.66343042e-04 6.65453074e-02 3.52346278e-02
 4.51051780e-02 1.61812298e-02 6.55339806e-03 1.28236246e-02
 2.45145631e-02 1.97006472e-02 3.24433657e-02 4.13025890e-02
 1.26779935e-01 4.32847896e-03 3.57038835e-01 3.09061489e-02
 2.06310680e-03 6.18932039e-03 2.38834951e-01 1.61003236e-01
 5.25889968e-04 3.35113269e-01 1.55339806e-02 4.12297735e-01
 3.87944984e-02 3.67313916e-02 1.31998382e-01 3.23624595e-04
 1.28236246e-01 8.48705502e-02 3.55582524e-02 5.19417476e-02
 7.08737864e-02 1.27750809e-01 5.98705502e-03 9.22734628e-02
 1.77184466e-02 1.07888350e-01 2.60922330e-02 5.16585761e-02
 6.68284790e-02 2.94296117e-01 3.01334951e-01 3.81877023e-02
 2.02305825e-01 1.30582524e-01 3.32928803e-02 1.11245955e-02
 3.08656958e-02 1.10720065e-01 9.95145631

# Robustness

We wish to see how robust the network is to perturbations around the average vector. Ideally for an epsilon ball of say 1% around the average, the classification does not change

In [ ]:
import numpy as np
from maraboupy import Marabou
from contextlib import redirect_stdout
import os


def epsilon_ball(model_path):
    net = Marabou.read_onnx(model_path)

# Load x_avg from file (or compute it in your current kernel)
    x_avg = np.load("x_avg.npy")

    # Access the input variable list; adjust indexing based on your network structure.
    # Here we assume net.inputVars[0][0] is an array of input variable IDs.
    inputs = net.inputVars[0][0].tolist()

    # Set epsilon as 5% of the average for each input dimension.
    for i, var in enumerate(inputs):
        # Compute epsilon_i as 5% of the absolute value of x_avg[i]
        epsilon_i = 0.01 * abs(x_avg[i])
        # Optionally, set a minimum epsilon (e.g., 0.01) if needed:
        # epsilon_i = max(0.05 * abs(x_avg[i]), 0.01)
        net.setLowerBound(var, x_avg[i] - epsilon_i)
        net.setUpperBound(var, x_avg[i] + epsilon_i)

    # Now you have set your input constraints to be within 5% of x_avg.
    with open(os.devnull, "w") as fnull:
        with redirect_stdout(fnull):
            exitCode, vals, stats = net.solve()
    print ("MODEL PATH:", model_path)
    if exitCode == "unsat":
        print("The network is robust in the 1% ε-ball around the average input.")
    else:
        print("Found a counterexample within the 1% ε-ball that changes the classification.")
        print("Counterexample:", vals)


In [ ]:
model_paths = ["model_4x100.onnx","model_4x200.onnx","model_4x300.onnx","model_6x100.onnx","model_6x200.onnx","model_6x300.onnx","model_8x100.onnx","model_8x200.onnx","model_8x300.onnx"]
for model_path in model_paths:
    epsilon_ball(model_path)

Instructions for updating:
non-resource variables are not supported in the long term
Engine::processInputQuery: Input query (before preprocessing): 401 equations, 910 variables
MODEL PATH: model_4x100.onnx
Found a counterexample within the 1% ε-ball that changes the classification.
Counterexample: {0: 43.80734217574289, 1: 186124.95, 2: 11.727773243208775, 3: 4046.2038808825405, 4: 193.05151511286826, 5: 45.92775666369085, 6: 0.04684223951026655, 7: 0.07947583216426476, 8: 0.0, 9: 0.6266254304297921, 10: 0.07853335033796709, 11: 0.09325851294477745, 12: 0.04546996556561663, 13: 0.0, 14: 0.02411554648641755, 15: 0.0078280831526591, 16: 0.007575564341282999, 17: 0.004166560387705649, 18: 0.0007728606045147303, 19: 0.0020201504910088, 20: 0.005152404030098202, 21: 0.00340900395357735, 22: 0.03345874250733325, 23: 0.045579645453386046, 24: 0.2804221400331591, 25: 0.03941589083025124, 26: 0.21148450452748374, 27: 0.12352888662160438, 28: 0.0, 29: 0.05448667261828848, 30: 0.17865960974365513